# 1. Importing libraries

In [1]:
import os 
import numpy
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 2. Reading dataset 

In [2]:
project_directory= r"E:\flight price prediction using sagemaker"
data_directory = "dataset"

In [3]:

def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(project_directory,data_directory,file_name)
    return pd.read_csv(file_path)


In [4]:
flights=get_data("flight_price")
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [5]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contains 10,683 rows and 11 features 
- columns 'Route' and 'Total_Stops' have missing value each 
- The data type of some features isn't appropriate 

# 3. Preliminary Analysis 

### 3.1 Checking Data types 

In [6]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [7]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

### 3.2 Check duplicates 

In [8]:
flights.duplicated().sum()

220

In [9]:
(
    flights
    .loc[flights.duplicated(keep=False)]
    .sort_values(["Airline","Date_of_Journey","Source","Destination"])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observations

- The type of 'Date_of_Journey' , 'Dep_Time' and  'Arrival_Time'  should be changes to datetime 
- The type of 'Duration' and 'Total_Stop' is mixed it should be numeric type.
- There are 220 duplicates these should be removed


# 4. Detailed Analysis 

### Airline

In [10]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- some of the values are inconsistent 

In [11]:
(
    
    flights
    .Airline
    .str.replace(" Premium economy", "")
    .str.replace(" Business", "")
    .str.title()
    .unique()
    
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

In [12]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [13]:
pd.to_datetime(flights.Date_of_Journey, dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### source 

In [14]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination 

In [15]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Departure time 

In [16]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [22]:
(
    flights
    .Dep_Time
    .loc[lambda ser:ser.str.contains("[^0-9:]")]

)

Series([], Name: Dep_Time, dtype: object)

In [24]:
pd.to_datetime(flights.Dep_Time).dt.time

C:\Users\Hp\AppData\Local\Temp\ipykernel_12224\1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arival_Time

In [27]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [31]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str.get(-1)
    .unique
)

<bound method Series.unique of 0        22 Mar
2        10 Jun
6        13 Mar
7        02 Mar
8        13 Mar
          ...  
10666    13 Jun
10667    13 Mar
10672    28 Jun
10673    28 May
10674    13 Mar
Name: Arrival_Time, Length: 4335, dtype: object>

## Duration

In [34]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [38]:
(
    flights
    .Duration 
    .loc[lambda ser : ~ser.str.contains("h")]
    #.unique()
)

6474    5m
Name: Duration, dtype: object

In [39]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


there is no point to keep the 6474 index 

In [55]:
(
    flights
    .Duration 
    .drop(index=6474)
    .str.split(" " , expand=True)
    .set_axis(["hour", "minutes"], axis=1 )
    .assign(
        hour=lambda df_:(
            
            df_
            .hour
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minutes=lambda df_:(
            df_
            .minutes
            .str.replace("m", "")
            .fillna(0)  
            .astype(int)
        )       
    )
    .sum(axis=1)
    #.astype(int)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [56]:
(
    flights
    .Duration 
    .drop(index=6474)
    .str.split(" " , expand=True)
    .set_axis(["hour", "minutes"], axis=1 )
    .assign(
        hour=lambda df_:(
            
            df_
            .hour
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minutes=lambda df_:(
            df_
            .minutes
            .str.replace("m", "")
            .fillna(0)  
            .astype(int)
        )       
    )
    .sum(axis=1)
    .rename("Duration_minutes")
    .to_frame()
    .join(flights.Duration.drop(index=6474))
    #.astype(int)
)

,Duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


### Total_Stops

In [61]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [65]:
import pandas as pd
(
    flights
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?", "",regex=True) # will either search for stop or stops 
    .pipe(lambda ser: pd.to_numeric(ser))
    
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

### Additional_Info 

In [72]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

# 5. Cleaning operations 

In [57]:
def convert_to_minutes(ser):
    return (
        ser 
        .str.split(" " , expand=True)
        .set_axis(["hour", "minutes"], axis=1 )
        .assign(
        hour=lambda df_:(
            
            df_
            .hour
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minutes=lambda df_:(
            df_
            .minutes
            .str.replace("m", "")
            .fillna(0)  
            .astype(int)
        )       
    )
    .sum(axis=1)
    )
    

In [84]:
def clean_data(df):
    return (
        df
        .drop(index=[6474])
        .assign(**{
            
            col: df[col].str.strip()
            for col in df.select_dtypes(include="O")
            
        })
        .rename(columns=str.lower)
        
        .assign(
            airline=lambda df_:(
                df_
                .airline
                .str.replace(" Premium economy", "")
                .str.replace(" Business", "")
                .str.title()
            ),
            date_of_Journey=lambda df_:pd.to_datetime(df_.date_of_journey, dayfirst=True),
            dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
            arrival_time=lambda df_:pd.to_datetime(df_.arrival_time).dt.time,
            duration =lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_:(
                
                df_
                .total_stops
                .replace("non-stop","0")
                .str.replace(" stops?", "",regex=True) # will either search for stop or stops 
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info=lambda df_:df_.additional_info.str.replace("No info", "No Info"),
            
        )
        .drop(columns=["route", "date_of_journey"])  # Drop the specified columns
        
    )

In [85]:
flights_cleaned= clean_data(flights)
flights_cleaned

C:\Users\Hp\AppData\Local\Temp\ipykernel_12224\3168329687.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
C:\Users\Hp\AppData\Local\Temp\ipykernel_12224\3168329687.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_:pd.to_datetime(df_.arrival_time).dt.time,


,airline,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price,date_of_Journey
0,Indigo,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897,2019-03-24
1,Air India,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662,2019-05-01
2,Jet Airways,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882,2019-06-09
3,Indigo,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218,2019-05-12
4,Indigo,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302,2019-03-01
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107,2019-04-09
10679,Air India,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145,2019-04-27
10680,Jet Airways,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229,2019-04-27
10681,Vistara,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648,2019-03-01


# 6. Split the Data 

In [86]:
flights_final = flights_cleaned.sample(1000)

In [87]:
X = flights_final.drop(columns="price")
y = flights_final.price.copy()

In [88]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


# 7. Export the subsets 

In [89]:
def export_data(X, y, name):
	file_name = f"{name}.csv"
	file_path = os.path.join(project_directory, data_directory, file_name)

	X.join(y).to_csv(file_path, index=False)

	return pd.read_csv(file_path).head()

In [90]:
export_data(X_train, y_train, "train")

,airline,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,date_of_Journey,price
0,Indigo,Banglore,Delhi,22:10:00,01:00:00,170,0.0,No Info,2019-04-18,4591
1,Jet Airways,Banglore,Delhi,17:45:00,20:45:00,180,0.0,No Info,2019-06-15,8016
2,Jet Airways,Delhi,Cochin,19:15:00,19:00:00,1425,1.0,No Info,2019-05-18,15554
3,Vistara,Banglore,New Delhi,07:00:00,09:40:00,160,0.0,No Info,2019-03-01,21520
4,Spicejet,Kolkata,Banglore,17:15:00,19:45:00,150,0.0,No Info,2019-03-03,6251


In [91]:
export_data(X_val, y_val, "val")

,airline,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,date_of_Journey,price
0,Multiple Carriers,Delhi,Cochin,11:40:00,19:15:00,455,1.0,No Info,2019-03-06,15353
1,Jet Airways,Banglore,Delhi,08:20:00,11:20:00,180,0.0,In-flight meal not included,2019-05-12,6144
2,Jet Airways,Kolkata,Banglore,18:55:00,20:45:00,1550,1.0,In-flight meal not included,2019-05-24,10844
3,Vistara,Kolkata,Banglore,17:00:00,10:45:00,1065,1.0,No Info,2019-06-06,9187
4,Air Asia,Kolkata,Banglore,22:50:00,01:20:00,150,0.0,No Info,2019-04-18,4409


In [92]:
export_data(X_test, y_test, "test")

,airline,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,date_of_Journey,price
0,Spicejet,Banglore,New Delhi,09:35:00,12:30:00,175,0.0,No Info,2019-03-03,7648
1,Indigo,Kolkata,Banglore,19:35:00,23:15:00,220,1.0,No Info,2019-05-24,4226
2,Indigo,Mumbai,Hyderabad,12:00:00,13:30:00,90,0.0,No Info,2019-05-06,2754
3,Jet Airways,Kolkata,Banglore,08:25:00,08:15:00,1430,1.0,No Info,2019-05-12,13941
4,Jet Airways,Mumbai,Hyderabad,02:55:00,04:20:00,85,0.0,No Info,2019-06-24,5678
